// voronoi  relaxation

//grid based hatching

// units are in mm

In [1]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line as draw_line
from skimage import data
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.transform import probabilistic_hough_line
import random 

# Load image"
# image_path = "/Users/nshelton/Pictures/gundam_title.png"
image_path = "/Users/nshelton/Pictures/power86.png"


image = cv2.imread(image_path)
print(image.shape)

# image[:,:,1] = image[:,:,0]
# image[:,:,2] = image[:,:,0]

image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_gray = cv2.resize(image_gray, (-1,-1), fx = 2, fy = 2)
 
_ , filtered =  cv2.threshold(image_gray, 150, 255, cv2.THRESH_BINARY_INV)
print(filtered)
# sigma = 2
# min_canny = 2
# max_canny = 100
# filtered = canny(image_gray, sigma, min_canny, max_canny).astype(np.uint8) * 255

cv2.imwrite("filtered.png", filtered)





(754, 1262, 3)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


True

In [24]:

remaining_ink = filtered.copy()
all_strokes_raster = np.ones(remaining_ink.shape) * 255

all_paths = []


def lookupxy(raster, xy) : return raster[int(xy[1]), int(xy[0])]

while len(all_paths) < 5000 and remaining_ink.sum() > 1000:
    print(remaining_ink.sum(), "\t", len(all_paths),  end="\t\t\t\r")
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(remaining_ink)

    def placeStroke(input_raster, start):
        img = input_raster.copy()
        path = []
        pos = np.array(start)

        window = 2

        max_val = lookupxy(input_raster, pos)

        while(max_val > 0):
            min_val, max_val, min_loc, max_loc  = cv2.minMaxLoc(img[pos[1] - window:pos[1] + window +1, pos[0] - window:pos[0] + window +1])
            pos += np.array(max_loc) - window 
            path.append(pos.copy())
            cv2.circle(img, pos, window-1, 0, -1)

        return path
    
    line_thickness = 5
    def rasterStroke(dims, path) :
        img = np.zeros(dims)
        for i in range(len(path)-1):
            a = (round(path[i][0]) , round(path[i][1]) )
            b = (round(path[i+1][0]) , round(path[i+1][1]) )
            cv2.line(img, a, b, 1, line_thickness, cv2.LINE_4)
            
        return img

    
    new_stroke = placeStroke(remaining_ink, max_loc)
    all_paths.append(new_stroke)
    stroke_raster = rasterStroke(image_gray.shape, new_stroke)

    remaining_ink = remaining_ink - stroke_raster * 255
    remaining_ink = np.clip(remaining_ink, 0, 255)
    all_strokes_raster = np.clip(all_strokes_raster.astype(np.int32) - stroke_raster.astype(np.int32) * 255, 0, 255).astype(np.uint8)

    # cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))
    # cv2.imwrite("remaining_ink.png", remaining_ink)
# cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))




In [20]:
import cv2
import numpy as np

from pathUtils import *



path_list = [[p.tolist() for p in path] for path in all_paths]

print(len(path_list))
# connect_close_paths(path_list)
path_list = [[p.tolist() for p in path] for path in all_paths]
render_img = plotPaths(path_list, 0.1)
cv2.imwrite("render.png", render_img)



456


True

In [21]:
from tqdm import tqdm



merge_threshold = 5


for i in tqdm(range(len(path_list))):
    len_before = len(path_list)
    path_list = merge_one_path_kd(path_list, merge_threshold)
    if len_before == len(path_list):
        break

print("merged to", len(path_list))




  3%|▎         | 13/456 [00:00<00:03, 119.19it/s]

merged to 443


In [22]:

path_list = close_loops(path_list)

print("closed loops", len(path_list))

path_list  = [np.array(path) for path in path_list]

for _ in range(1):
    path_list = smooth_all_2(path_list, 0.01)

print("smoothed 1", len(path_list))

path_list = simplify_all(path_list, 0.001)

print("simplify_all 1", len(path_list))

for _ in range(1):
    path_list = smooth_all_2(path_list, 0.1)

print("smoothed 2", len(path_list))

path_list = simplify_all(path_list, 0.1)
print("simplify 2", len(path_list))





closed loops 443
smoothed 1 443
simplify_all 1 443
smoothed 2 443
simplify 2 443


In [23]:

render_img = plotPaths(path_list, 0.2)
cv2.imwrite("render.png", render_img)


True

In [17]:
 
import json 
import uuid

projects_folder = "/Users/nshelton/Hephaestus/projects"

def writePlot(paths):
    dictionary = {
        "created_time": "4/7/2024 4:42:27 PM",
        "modified_time": "4/7/2024 6:27:36 PM",
        "camera_position": [100, 100],
        "zoom": 10.702,
        "aspect": 0.563,
        "dom_element": {},
        "plot_models": [],
    }

    dictionary["plot_models"].append(
        {
            "position": {"x": 0, "y": 0},
            "paths": [path.tolist() for path in path_list],
            "scale": 0.03,
            "id": str(uuid.uuid4()),
            "state": "none",
        }
    )


    with open(projects_folder + "/circuit.json", "w") as outfile:
        
        json.dump(dictionary, outfile)


writePlot(path_list)

